In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split



import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import collections as col
import csv
from datetime import datetime
import os.path
from typing import List

In [76]:



def load_images_from_folder(images_FP7):
    images = []
    
    for bild in os.listdir(images_FP7):
    
        #print(bild)
        name = bild.split('_')
        lw_in_name = name[3].split('.')
        lenkwinkel = int(lw_in_name[0])
        # print(name)
        # print(lenkwinkel)
        img = cv.imread(os.path.join(images_FP7,bild))
        image_lw = [img, lenkwinkel]
        if img is not None:
            images.append(image_lw)
            
    return images

    #Lenkwinkel_Dict:
    #path_dir: str = r"/home/imm-81/c2c/camp2code-project_phase_2/Code/images_FP7"

    #content_dir: List[str] = os.listdir(path_dir)

Bild_list_array = load_images_from_folder('images_FP7')
print(len(Bild_list_array))
bild_act = Bild_list_array[0]
print(bild_act[1])
cv.imshow('test', bild_act[0])
cv.waitKey(0)
cv.destroyAllWindows()





157
118


In [ ]:
X_train, X_test, y_train, x_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

cnn_model = Sequential([
    Conv2D(filters=32, kernel_size=3,activation='relu',input_shape = image_shape),
    MaxPooling2D(pool_size=2),
    Dropout(0.2),
    Flatten(),
    Dense(128,activation='relu'),
    Dense(10,activation = 'softmax') # Softmax-Schicht
])